This comes up in discussions a lot so here is a notebook that walks through how MAGIC adds noise to images. (Version 2.2)

In [2]:
import os

from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

(Step through buildfgssteps.create_img_arrays)

In [ ]:
def create_img_arrays(self, step, config_ini):
        """Create the needed image arrays for the given step.

        Possible arrays (created as class attributes):
            - time_normed_im : the "sky" image, or the time-normalized
                image (in counts)
            - bias : the FGS bias used to simulate the image; includes
                0th read bias structure and KTC "shot" noise. An array
                of size (nramps x nreads) x n_rows x n_columns
            - cds : subtracts the zeroth read from the first read
            - strips : divides a full-frame array into 36 strips of
                size 64 x 2048

        If creating LOSTRK images, the final "image" array will be
        normalized and resized for use in the FGSES

        Parameters
        ----------
        step : str
            Name of step within the config file ('{step}_dict')
        config_ini : : obj
            Object containing all parameters from the given config file

        Returns
        -------
        image : 2-D numpy array
            The final image including any necessary detector effects,
            either full-frame or the appropriately sized subarray, in counts.
        """

        # Create the time-normalized image (will be in counts, where the
        # input_im is in counts per second)
        self.time_normed_im = self.input_im * config_ini.getfloat(step, 'tframe')

        # Add the bias, and build the array of reads with noisy data
        if config_ini.getboolean(step, 'bias'):
            # Get the bias ramp
            nramps = config_ini.getint(step, 'nramps')
            det_eff = detector_effects.FGSDetectorEffects(
                self.guider, self.xarr, self.yarr, self.nreads, nramps,
                config_ini.getint(step, 'imgsize')
            )
            self.bias = det_eff.add_detector_effects() ### INTERJECT HERE

            # Take the bias and add a noisy version of the input image
            # (specifically Poisson noise), adding signal over each read
            image = np.copy(self.bias)
            signal_cube = [self.time_normed_im] * nramps
            positive_signal_cube = np.copy(signal_cube)
            positive_signal_cube[positive_signal_cube < 0] = 0

            # First read
            # Calculate how much signal should be in the first read
            i_read = 0
            n_drops_before_first_read = int(config_ini.getfloat(step, 'ndrops1'))
            n_frametimes_in_first_read = n_drops_before_first_read + 1
            # Add signal to every first read
            noisy_signal_cube = np.random.poisson(positive_signal_cube)
            image[i_read::self.nreads] += n_frametimes_in_first_read * noisy_signal_cube

            # Second read
            # Calculate how much signal should be in the second read
            i_read = 1
            n_drops_before_second_read = int(config_ini.getfloat(step, 'ndrops2'))
            n_frametimes_in_second_read = n_frametimes_in_first_read + n_drops_before_second_read + 1
            # Add signal to every second read
            noisy_signal_cube = np.random.poisson(positive_signal_cube)
            image[i_read::self.nreads] += n_frametimes_in_second_read * noisy_signal_cube

        else:
            # In the case of LOSTRK, just return one frame with one
            # frame readout worth of signal
            self.bias = None
            image = self.time_normed_im

        # Cut any pixels over saturation or under zero
        image = utils.correct_image(image, upper_threshold=65535, upper_limit=65535)

        # Create the CDS image by subtracting the first read from the second
        # read, for each ramp
        if config_ini.getboolean(step, 'cdsimg'):
            self.cds = create_cds(image, step, config_ini)
        else:
            self.cds = None

        # If in ID, split the full-frame image into strips
        if config_ini.getboolean(step, 'stripsimg'):
            self.strips = create_strips(image,
                                        config_ini.getint(step, 'imgsize'),
                                        config_ini.getint(step, 'nstrips'),
                                        config_ini.getint(step, 'nramps'),
                                        self.nreads,
                                        config_ini.getint(step, 'height'),
                                        self.yoffset,
                                        config_ini.getint(step, 'overlap'))

        # Modify further for LOSTRK images (that will be run in FGSES)
        if self.step == 'LOSTRK':
            # Resize image array to oversample by 6 (from 43x43 to 255x255)
            image = image.repeat(6, axis=0)
            image = image.repeat(6, axis=1)
            image = image[1:-2, 1:-2]

            # Normalize to a count sum of 1000
            image = image / np.sum(image) * 1000

        return image

In [3]:
step = "ID"
repo_location = '/Users/kbrooks/git_repos/jwst-magic-fork/jwst_magic' #TODO: replace with generalization
config_ini = os.path.join(repo_location, 'config.ini')

In [ ]:
# make a dummy image
input_image = 